In [ ]:
import numpy as np
from scipy.fft import fft, ifft, fftfreq
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
def spectral_pooling(x, dt, max_freq):
    srate = 1 / dt
    xf = fft(x)
    n_samples = x.size
    freq = fftfreq(n_samples, dt)
    xf_trunc = xf[np.abs(freq) <= max_freq]
    x_sub = ifft(xf_trunc) * (max_freq / srate *  2)
    t_sub = np.linspace(0, dt * n_samples, x_sub.size)
    return t_sub, np.real(x_sub)

In [ ]:
tend = 20
srate = 40
dt = 1/srate
t = np.r_[0 : tend : dt]
F = [0.2, 1, 2, 5, 7]
x = np.array([np.cos(2 * np.pi * f * t) for f in F]).sum(axis=0)
eta = 1 * np.random.uniform(size=x.shape)
x += eta
t_pooled, x_pooled = spectral_pooling(x, dt, srate/4)

In [ ]:
fig,ax = plt.subplots(1, 1)
ax.plot(t, x, 'k', lw=1, label='Original')
ax.plot(t_pooled, x_pooled, 'r', lw=1, label='Downsampled')
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.legend(loc='lower right')
ax.set_xlabel('Time [s]')
fig.tight_layout()